In [17]:
import json
import math
import os
import random
import re
import unicodedata
import uuid
from dataclasses import dataclass
from datetime import date, datetime, time, timedelta
from pathlib import Path
import numpy as np
import pandas as pd

### BLOQUE 1

Definicion de constantes, la semilla, el rango de fechas, número de empresas, usuarios por empresa y tickets por usuario, así como rutas de archivos y parámetros de simulación.

In [ ]:
PY_SEED = 42
np.random.seed(PY_SEED)
random.seed(PY_SEED)

YEAR = 2025
FECHA_INI = date(YEAR, 1, 1)
FECHA_FIN = date(YEAR, 7, 31)
assert FECHA_INI <= FECHA_FIN, "FECHA_INI debe ser <= FECHA_FIN"

N_EMPRESAS_TRANSP = 3
USUARIOS_POR_EMPRESA = 3
TICKETS_POR_USUARIO = 50
PESO_TARJETA = 0.85  

STATION_OFFSET_MIN, STATION_OFFSET_MAX = -0.02, 0.02
DAILY_NOISE_MIN, DAILY_NOISE_MAX       = -0.01, 0.01
assert STATION_OFFSET_MIN <= STATION_OFFSET_MAX, "Rango station offset inválido"
assert DAILY_NOISE_MIN   <= DAILY_NOISE_MAX,   "Rango daily noise inválido"

LITROS_MIN, LITROS_MAX, LITROS_MODA = 10.0, 80.0, 35.0
assert LITROS_MIN <= LITROS_MODA <= LITROS_MAX, "Moda de litros fuera de rango"

PRODUCTOS = [
    "Gasolina 95 E5",
    "Gasolina 98 E5",
    "Gasóleo A",
    "Gasóleo Premium",
]


DATA_DIR = Path(r"data")
DATA_DIR.mkdir(parents=True, exist_ok=True)  

PATH_ESTACIONES = DATA_DIR / "EstacionesDeServicio.csv"
PATH_NIFS       = DATA_DIR / "nif_empresas_gasolineras_es.csv"
PATH_PRECIOS    = DATA_DIR / "PreciosProvincia.csv"


OUT_DIR = DATA_DIR / "tickets"
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_JSONL = OUT_DIR / "tickets_sinteticos.jsonl"
OUT_JSON  = OUT_DIR / "tickets_sinteticos.json"

### BLOQUE 2

Definicion de funciones auxiliares para:

Normalizar texto (norm_txt).

Redondear números (round2, round3).

Generar fechas y horas aleatorias (random_fecha, random_hora).

Generar litros de combustible según distribución triangular (random_litros).

Elegir método de pago aleatoriamente (elegir_metodo_pago).

Convertir fechas y horas a string (str_fecha, str_hora).

In [19]:
def norm_txt(x: str) -> str:
    if pd.isna(x): return ""
    x = str(x).strip()
    x = "".join(c for c in unicodedata.normalize("NFD", x) if unicodedata.category(c) != "Mn")
    return x.lower()

def round2(x: float) -> float:
    return float(np.round(x + 1e-12, 2))

def round3(x: float) -> float:
    return float(np.round(x + 1e-12, 3))

def random_fecha(fecha_ini: date, fecha_fin: date) -> date:
    delta = (fecha_fin - fecha_ini).days
    return fecha_ini + timedelta(days=int(np.random.randint(0, delta + 1)))

def random_hora() -> time:
    return (datetime.min + timedelta(seconds=int(np.random.randint(0, 24*3600)))).time()

def random_litros() -> float:
    return round2(np.random.triangular(LITROS_MIN, LITROS_MODA, LITROS_MAX))

def elegir_metodo_pago() -> str:
    return "Tarjeta crédito" if random.random() < PESO_TARJETA else "Efectivo"

def str_fecha(d: date) -> str:
    return d.strftime("%Y-%m-%d")

def str_hora(t: time) -> str:
    return t.strftime("%H:%M:%S")

### BLOQUE 3

Bloque grande de carga y limpieza de CSVs:

Lectura flexible de CSVs de estaciones, NIFs y precios.

Normalización de nombres de provincias, empresas, marcas y productos.

Detección de columnas relevantes y creación de IDs de estación.

Conversión de lat/lon a floats y validación de coordenadas.

Construcción de diccionarios de precios por provincia, producto y mes (precios_idx) y precios nacionales (precios_nac_idx).

In [20]:
def load_csv_guess(path: Path):
    encodings = ["cp1252", "utf-8", "latin1"]
    seps = [",", ";", "|", "\t"]
    last_err = None
    for enc in encodings:
        for sep in seps:
            try:
                df = pd.read_csv(path, encoding=enc, sep=sep)
                if df.shape[1] >= 2:
                    return df
            except Exception as e:
                last_err = e
                continue
    raise RuntimeError(f"No pude leer {path} — último error: {last_err}")

def _simplify(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", norm_txt(s))

def to_float_locale(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series.astype(str).str.replace(",", ".", regex=False), errors="coerce")

def prov_key(x: str) -> str:
    s = norm_txt(x)
    if "/" in s: s = s.split("/")[0].strip()
    repl = {
        "vizcaya":"bizkaia",
        "guipuzcoa":"gipuzkoa",
        "guipuzcoa.":"gipuzkoa",
        "la coruna":"a coruna",
        "coruna":"a coruna",
        "coruña":"a coruna",
        "orense":"ourense",
        "lerida":"lleida",
        "gerona":"girona",
        "valencia/valencia":"valencia",
        "alicante/alacant":"alicante",
        "castello":"castellon",
        "santa cruz de tenerife":"santa cruz de tenerife",
        "tenerife":"santa cruz de tenerife",
        "araba alava":"alava",
        "araba/alava":"alava",
        "alava":"alava",
    }
    s = repl.get(s, s)
    return s

raw_est = load_csv_guess(PATH_ESTACIONES).copy()
est = raw_est.copy()
est.columns = [norm_txt(c) for c in est.columns]

for cand in ["ideess","id","idestacion","id_estacion","codigo","id_eess"]:
    if cand in est.columns:
        col_id = cand; break
else:
    col_id = "_idgen"; est[col_id] = np.arange(1, len(est)+1)

for cand in ["rotulo","estacion","nombre","razon_social","razonsocial","marca","rótulo"]:
    if cand in est.columns:
        col_nombre = cand; break
else:
    col_nombre = None

for cand in ["grupo","empresa","rotulo","marca","compania","compañia","rótulo"]:
    if cand in est.columns:
        col_grupo = cand; break
else:
    col_grupo = None

for cand in ["provincia","desc_provincia","nomprovincia","prov_name","province","prov","provincia_iso","provincia_nombre"]:
    if cand in est.columns:
        col_prov = cand; break
else:
    raise ValueError("No encuentro columna de provincia en EstacionesDeServicio")

for cand in ["municipio","poblacion","localidad","ciudad","town"]:
    if cand in est.columns:
        col_muni = cand; break
else:
    col_muni = None

for cand in ["direccion","dirección","address","dir","calle"]:
    if cand in est.columns:
        col_dir = cand; break
else:
    col_dir = None

col_lat = None
col_lon = None
for c in est.columns:
    sc = _simplify(c)
    if col_lat is None and ("latitud" in sc or sc.endswith("lat") or sc == "lat"):
        col_lat = c
    if col_lon is None and ("longitud" in sc or sc.endswith("lon") or sc in ("lon","long","lng")):
        col_lon = c

lat_series = to_float_locale(est[col_lat]) if col_lat else pd.Series(np.nan, index=est.index)
lon_series = to_float_locale(est[col_lon]) if col_lon else pd.Series(np.nan, index=est.index)

if (lat_series.lt(-20).mean() > 0.2) or (lon_series.gt(20).mean() > 0.2):
    lat_series, lon_series = lon_series, lat_series

est_std = pd.DataFrame({
    "id_estacion": est[col_id].astype(str),
    "nombre": est[col_nombre].astype(str) if col_nombre else "",
    "grupo": est[col_grupo].astype(str) if col_grupo else "",
    "provincia": est[col_prov].astype(str),
    "municipio": est[col_muni].astype(str) if col_muni else "",
    "direccion": est[col_dir].astype(str) if col_dir else "",
    "lat": lat_series,
    "lon": lon_series,
})

est_std["provincia_norm"] = est_std["provincia"].map(prov_key)
est_std["grupo_norm"] = est_std["grupo"].map(norm_txt)

if est_std["lat"].notna().mean() > 0.8 and est_std["lon"].notna().mean() > 0.8:
    est_std = est_std.dropna(subset=["lat","lon"]).reset_index(drop=True)

raw_nifs = load_csv_guess(PATH_NIFS).copy()
nifs = raw_nifs.copy()
nifs.columns = [norm_txt(c) for c in nifs.columns]

def simplify_colname(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", norm_txt(s))

cols = list(nifs.columns)
cols_simpl = {c: simplify_colname(c) for c in cols}
company_keys = ["compania","razonsocial","empresa","grupo","sociedad","razon"]
brand_keys   = ["marca","rotulo","brand"]
nif_keys     = ["nif","cif","nifcif","cifnif"]

def pick_col(keys):
    for c in cols:
        sc = cols_simpl[c]
        if any(k in sc for k in keys):
            return c
    return None

col_emp   = pick_col(company_keys)
col_marca = pick_col(brand_keys)
col_nif   = pick_col(nif_keys)
if not col_nif or not (col_emp or col_marca):
    raise ValueError("nif_empresas_gasolineras_es.csv debe tener una columna con NIF/CIF y otra con Compañía/Grupo/Marca.")

map_grupo_to_nif = {}
def add_alias(name, nif):
    k = norm_txt(str(name)).strip()
    if k: map_grupo_to_nif[k] = str(nif).strip().upper()
def split_aliases(text):
    parts = re.split(r"[;,/|&]+", str(text))
    return [p.strip() for p in parts if str(p).strip()]

for _, r in nifs.iterrows():
    nif_val = r[col_nif]
    if pd.isna(nif_val) or str(nif_val).strip() == "": continue
    if col_emp and pd.notna(r[col_emp]): add_alias(r[col_emp], nif_val)
    if col_marca and pd.notna(r[col_marca]):
        for alias in split_aliases(r[col_marca]): add_alias(alias, nif_val)

raw_pre = load_csv_guess(PATH_PRECIOS).copy()
pre = raw_pre.copy()
pre.columns = [norm_txt(c) for c in pre.columns]

col_pprov = next((c for c in ["provincia","province","prov"] if c in pre.columns), None)
col_prod  = next((c for c in ["producto","product","carburante","fuel","tipo"] if c in pre.columns), None)
col_mes   = next((c for c in ["mes","month"] if c in pre.columns), None)
col_anio  = next((c for c in ["anio","año","year"] if c in pre.columns), None)
col_fecha = next((c for c in ["fecha_precio","fecha","date","f_precio","fecha_pvp"] if c in pre.columns), None)
precio_candidatas = ["promedio_de_pvp_diario_cubo","pvp","pvp_medio","precio","precio_medio","price","promedio_de_pai_diario_cubo","pai","precio_pai"]
col_prec = next((c for c in precio_candidatas if c in pre.columns), None)
if not col_prec:
    for c in pre.columns:
        s = pd.to_numeric(pre[c].astype(str).str.replace(",", ".", regex=False), errors="coerce")
        if s.notna().mean() > 0.6: col_prec = c; break
if not (col_pprov and col_prod and (col_mes or col_fecha) and col_prec):
    raise ValueError("PreciosProvincia.csv: necesito provincia, producto, precio y mes o fecha_precio.")

def canon_producto(x: str) -> str:
    s = norm_txt(x).replace("+","plus")
    if "98" in s: return "Gasolina 98 E5"
    if "premium" in s or "plus" in s or "a+" in s: return "Gasóleo Premium"
    if "gasoil" in s or "gasoleo" in s or "gasóleo" in s or "diesel" in s or "habitual" in s: return "Gasóleo A"
    return "Gasolina 95 E5"

if col_fecha:
    pre["_fecha_dt"] = pd.to_datetime(pre[col_fecha], errors="coerce", dayfirst=True, infer_datetime_format=True)
    if col_anio:
        pre = pre.loc[pd.to_numeric(pre[col_anio], errors="coerce").fillna(pre["_fecha_dt"].dt.year) == YEAR].copy()
    else:
        pre = pre.loc[pre["_fecha_dt"].dt.year == YEAR].copy()
    pre["_mes"]        = pre["_fecha_dt"].dt.month
    pre["_prov_norm"]  = pre[col_pprov].map(prov_key)
    pre["_prod_canon"] = pre[col_prod].map(canon_producto)
    pre["_precio"]     = pd.to_numeric(pre[col_prec].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    parsed = pre.groupby(["_prov_norm","_prod_canon","_mes"], as_index=False)["_precio"].mean()
else:
    MES_MAP = {
        "1":1,"01":1,"enero":1,"ene":1,"january":1,
        "2":2,"02":2,"febrero":2,"feb":2,"february":2,
        "3":3,"03":3,"marzo":3,"mar":3,"march":3,
        "4":4,"04":4,"abril":4,"abr":4,"april":4,
        "5":5,"05":5,"mayo":5,"may":5,
        "6":6,"06":6,"junio":6,"jun":6,"june":6,
        "7":7,"07":7,"julio":7,"jul":7,"july":7,
        "8":8,"08":8,"agosto":8,"ago":8,"august":8,
        "9":9,"09":9,"septiembre":9,"setiembre":9,"sep":9,"sept":9,"september":9,
        "10":10,"octubre":10,"oct":10,"october":10,
        "11":11,"noviembre":11,"nov":11,"november":11,
        "12":12,"diciembre":12,"dic":12,"dec":12,"december":12,
    }
    parsed = pre[[col_pprov, col_prod, col_mes, col_prec] + ([col_anio] if col_anio else [])].copy()
    parsed["_prov_norm"]  = parsed[col_pprov].map(prov_key)
    parsed["_prod_canon"] = parsed[col_prod].map(canon_producto)
    parsed["_mes"]        = parsed[col_mes].astype(str).map(lambda x: MES_MAP.get(norm_txt(x), np.nan)).astype(float)
    parsed["_precio"]     = pd.to_numeric(parsed[col_prec].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    if col_anio:
        parsed = parsed.loc[pd.to_numeric(parsed[col_anio], errors="coerce") == YEAR].copy()

parsed["_mes"]    = pd.to_numeric(parsed["_mes"], errors="coerce")
parsed["_precio"] = pd.to_numeric(parsed["_precio"], errors="coerce")
parsed = parsed.loc[parsed["_mes"].isin(range(1,8))].copy()
parsed = parsed.dropna(subset=["_prov_norm","_prod_canon","_mes","_precio"])
parsed["_mes"] = parsed["_mes"].astype(int)

precios_idx = parsed.set_index(["_prov_norm","_prod_canon","_mes"])["_precio"].to_dict()
NACIONAL_KEYS = {norm_txt(x) for x in ["España","Total Nacional","Nacional","Media Nacional","Total"]}
hay_nacional = any(k in NACIONAL_KEYS for k in parsed["_prov_norm"].dropna().unique())
if hay_nacional:
    pre_nac = parsed[parsed["_prov_norm"].isin(NACIONAL_KEYS)].copy()
    precios_nac_idx = pre_nac.set_index(["_prod_canon","_mes"])["_precio"].to_dict()
else:
    precios_nac_idx = {}

C:\Users\Jon\AppData\Local\Temp\ipykernel_3036\3662935373.py:186: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pre["_fecha_dt"] = pd.to_datetime(pre[col_fecha], errors="coerce", dayfirst=True, infer_datetime_format=True)


### BLOQUE 4

Bloque de generación de CIFs aleatorios:

Define grupos de letras para el primer carácter según la normativa de CIF.

Calcula la suma de dígitos pares e impares para el dígito de control.

Devuelve un CIF completo aleatorio (cif_generate()).

In [21]:
CONTROL_LETTERS = "JABCDEFGHI"
LETTER_GROUP = set(list("PQRSNW"))
DIGIT_GROUP  = set(list("ABEH"))
ANY_GROUP    = set(list("CDFGJUVXYZ"))

def _sum_digits(n: int) -> int:
    return n if n < 10 else n//10 + n%10

def cif_generate() -> str:
    first = random.choice(list(LETTER_GROUP | DIGIT_GROUP | ANY_GROUP))
    digits = [random.randint(0,9) for _ in range(7)]
    sum_even = digits[1] + digits[3] + digits[5]
    sum_odd = sum(_sum_digits(2*d) for d in (digits[0], digits[2], digits[4], digits[6]))
    total = sum_even + sum_odd
    cd_num = (10 - (total % 10)) % 10
    if first in LETTER_GROUP:
        control = CONTROL_LETTERS[cd_num]
    elif first in DIGIT_GROUP:
        control = str(cd_num)
    else:
        control = str(cd_num)
    body = "".join(str(d) for d in digits)
    return f"{first}{body}{control}"

### BLOQUE 5

Definicion de NIF de grupo:

Toma el nombre de la empresa o grupo.

Busca un NIF/CIF en el diccionario map_grupo_to_nif.

Si no hay NIF válido, genera uno aleatorio usando cif_generate().

In [22]:
def nif_de_grupo(grupo_texto: str) -> str:
    key = norm_txt(grupo_texto)
    nif = map_grupo_to_nif.get(key, None)
    if isinstance(nif, str) and len(nif.strip()) >= 8:
        return nif.strip().upper()
    return cif_generate()

### BLOQUE 6

Generacion de funciones de precio:

station_offset(id_estacion): genera un offset fijo por estación usando hash.

precio_base_mes(prov_norm, prod_canon, mes): obtiene el precio medio mensual por provincia y producto, o nacional si no hay provincial.

precio_diario(prov_norm, prod_canon, fecha, id_estacion): añade offset de estación y ruido diario al precio base.

In [23]:
def station_offset(id_estacion: str) -> float:
    rnd = random.Random(hash(id_estacion) & 0xffffffff)
    return float(rnd.uniform(STATION_OFFSET_MIN, STATION_OFFSET_MAX))

def precio_base_mes(prov_norm: str, prod_canon: str, mes: int) -> float | None:
    key = (prov_norm, prod_canon, mes)
    if key in precios_idx and pd.notna(precios_idx[key]): return float(precios_idx[key])
    key_n = (prod_canon, mes)
    if key_n in precios_nac_idx and pd.notna(precios_nac_idx[key_n]): return float(precios_nac_idx[key_n])
    return None

def precio_diario(prov_norm: str, prod_canon: str, fecha: date, id_estacion: str) -> float:
    base = precio_base_mes(prov_norm, prod_canon, fecha.month)
    if base is None or math.isnan(base):
        candidatos = [v for (p, pr, m), v in precios_idx.items() if pr == prod_canon and m == fecha.month and pd.notna(v)]
        base = float(np.mean(candidatos)) if candidatos else 1.50
    off = station_offset(id_estacion)
    noise = random.uniform(DAILY_NOISE_MIN, DAILY_NOISE_MAX)
    return round3(max(0.5, base + off + noise))

### BLOQUE 7

Bloque donde se crean las empresas y los usuarios:

Se definen 9 usuarios por empresa --> Conseguimos asi que haya mas tipos diferentes de producto

Cada usuario tiene un ID secuencial global, tipo EMP01-U1….

Creamos un diccionario usuario_producto para garantizar que cada usuario siempre reposte el mismo combustible y asi haya consistencia.

In [24]:
@dataclass
class Empresa:
    id: str
    nombre: str

@dataclass
class Usuario:
    id: str
    empresa_id: str
    nombre: str

empresas = [Empresa(id=f"EMP{i+1:03d}", nombre=f"Transporte_{i+1:02d} S.L.") for i in range(N_EMPRESAS_TRANSP)]
usuarios = []
for e in empresas:
    for j in range(USUARIOS_POR_EMPRESA):
        usuarios.append(Usuario(id=f"{e.id}-U{j+1:03d}", empresa_id=e.id, nombre=f"Usuario_{j+1:02d}_{e.id}"))

USUARIOS_POR_EMPRESA = 9


usuarios = []
usuario_producto = {}
usuario_cont = 1
for e in empresas:
    productos_asignados = random.choices(PRODUCTOS, k=USUARIOS_POR_EMPRESA)
    for j in range(USUARIOS_POR_EMPRESA):
        u_id = f"{e.id}-U{usuario_cont}"
        usuarios.append(Usuario(id=u_id, empresa_id=e.id, nombre=f"Usuario_{usuario_cont:02d}_{e.id}"))
        usuario_producto[u_id] = productos_asignados[j]
        usuario_cont += 1

### BLOQUE 8

Este bloque prepara el pool de estaciones para la generación de tickets:

EST_POOL = est_std.reset_index(drop=True).copy(): resetea el índice y crea un DataFrame limpio de estaciones.

Convierte las coordenadas lat y lon a tipo numérico, reemplazando comas por puntos.

Comprueba que haya al menos una estación (assert len(EST_POOL) > 0).

In [25]:
EST_POOL = est_std.reset_index(drop=True).copy()
EST_POOL["lat"] = pd.to_numeric(EST_POOL["lat"].astype(str).str.replace(",", ".", regex=False), errors="coerce")
EST_POOL["lon"] = pd.to_numeric(EST_POOL["lon"].astype(str).str.replace(",", ".", regex=False), errors="coerce")
assert len(EST_POOL) > 0

### BLOQUE 9

Bloque de cálculo de importes.

Se define el 21 % de IVA.

calcular_importes(litros, precio_unit) devuelve un diccionario con:

    precio_unitario

    importe_total

    base_imponible

    iva

La función ajusta el IVA para que base + iva = total tras redondear, evitando diferencias por decimales.

In [26]:
IVA_TIPO = 0.21

def calcular_importes(litros: float, precio_unit: float) -> dict:
    total = round2(litros * precio_unit)
    base = round2(total / (1 + IVA_TIPO))
    iva  = round2(total - base)
    if round2(base + iva) != total:
        diff = round2(total - (base + iva))
        iva = round2(iva + diff)
    return {"precio_unitario": precio_unit, "importe_total": total, "base_imponible": base, "iva": iva}

### BLOQUE 10

Bloque clave de generación de tickets:

Aseguramos que el usuario siempre reposte el mismo tipo de de combustible

Los IDs de ticket (idTicket) se generan con UUID

Se selecciona una estación aleatoria de EST_POOL.

Se calculan importes, IVA, precio unitario y demás campos.

In [27]:
def _to_float_locale(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip().replace(",", ".")
    try:
        return float(s)
    except Exception:
        return np.nan

def _coords_from_row(row):
    lat = _to_float_locale(row.get("lat", np.nan))
    lon = _to_float_locale(row.get("lon", np.nan))

    def _ok(la, lo):
        return (27.0 <= la <= 44.5) and (-20.0 <= lo <= 5.5)

    if not _ok(lat, lon) and _ok(lon, lat):
        lat, lon = lon, lat

    if not _ok(lat, lon):
        return np.nan, np.nan
    return lat, lon

def generar_ticket(empresa: Empresa, usuario: Usuario) -> dict:
    f = random_fecha(FECHA_INI, FECHA_FIN)
    h = random_hora()

    lat, lon = np.nan, np.nan
    attempts = 0
    row = None
    while attempts < 5 and (pd.isna(lat) or pd.isna(lon)):
        row = EST_POOL.sample(1).iloc[0]
        lat, lon = _coords_from_row(row)
        attempts += 1

    producto = usuario_producto[usuario.id]
    prov_norm = row["provincia_norm"]
    punit = precio_diario(prov_norm, producto, f, row["id_estacion"])

    litros = random_litros()
    metodo = elegir_metodo_pago()

    imp = calcular_importes(litros, punit)

    nif_empresa_gasolinera = nif_de_grupo(row["grupo"])

    ticket = {
        "idTicket": f"T-{uuid.uuid4().hex[:12].upper()}",
        "idEmpresa": empresa.id,
        "empresaNombre": empresa.nombre,
        "idUsuario": usuario.id,
        "fechaEmision": str_fecha(f),
        "horaEmision": str_hora(h),
        "metodoPago": metodo,
        "estacion": {
            "id": row["id_estacion"],
            "nombre": row["nombre"],
            "provincia": row["provincia"],
            "municipio": row["municipio"],
            "direccion": row["direccion"],
            "lat": None if pd.isna(lat) else float(lat),
            "lon": None if pd.isna(lon) else float(lon),
            "grupo": row["grupo"],
            "nifEmpresa": nif_empresa_gasolinera
        },
        "lineas": [
            {
                "producto": producto,
                "litros": litros,
                "precioUnitario": imp["precio_unitario"],
                "importe": imp["importe_total"]
            }
        ],
        "baseImponible": imp["base_imponible"],
        "iva": imp["iva"],
        "total": imp["importe_total"],
        "moneda": "EUR",
        "tipoDocumento": "Factura simplificada"
    }
    return ticket

### BLOQUE 10

Bloque donde definimos como generar todos los tickets sintéticos del notebook:

Función generar_todos():

    Inicializa una lista vacía tickets.

    Recorre todas las empresas (for emp in empresas:).

    Para cada empresa, obtiene sus usuarios (us_emp = [u for u in usuarios if u.empresa_id == emp.id]).

    Para cada usuario de esa empresa, genera TICKETS_POR_USUARIO tickets llamando a la función generar_ticket(emp, u).

    Añade cada ticket a la lista tickets.

    Devuelve la lista completa de tickets.

Llamada a la funcion y generacion

In [28]:
def generar_todos() -> list[dict]:
    tickets = []
    for emp in empresas:
        us_emp = [u for u in usuarios if u.empresa_id == emp.id]
        for u in us_emp:
            for _ in range(TICKETS_POR_USUARIO):
                tickets.append(generar_ticket(emp, u))
    return tickets

tickets = generar_todos()

Verificacion de la ruta de salida JSON

In [29]:
from pathlib import Path, PureWindowsPath
p = Path(OUT_JSON)
print("path:", p, "| exists:", p.exists(), "| is_dir:", p.is_dir(), "| is_file:", p.is_file())

path: data\tickets\tickets_sinteticos.json | exists: True | is_dir: False | is_file: True


Guardado de los tickets

In [30]:
with open(OUT_JSONL, "w", encoding="utf-8") as f:
    for tk in tickets:
        f.write(json.dumps(tk, ensure_ascii=False) + "\n")

with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(tickets, f, ensure_ascii=False, indent=2)

OUT_JSONL, OUT_JSON

(WindowsPath('data/tickets/tickets_sinteticos.jsonl'),
 WindowsPath('data/tickets/tickets_sinteticos.json'))

Control de calidad de los archivos generados

In [31]:
df_chk = pd.DataFrame([{
    "empresa":  t.get("idEmpresa"),
    "usuario":  t.get("idUsuario"),
    "fecha":    t.get("fechaEmision"),
    "hora":     t.get("horaEmision"),
    "metodo":   t.get("metodoPago"),

    "producto": t["lineas"][0].get("producto") if t.get("lineas") else None,
    "precio":   t["lineas"][0].get("precioUnitario") if t.get("lineas") else None,
    "litros":   t["lineas"][0].get("litros") if t.get("lineas") else None,
    "total":    t.get("total"),

    "est_id":        t.get("estacion", {}).get("id"),
    "est_nombre":    t.get("estacion", {}).get("nombre"),
    "est_grupo":     t.get("estacion", {}).get("grupo"),
    "est_nif":       t.get("estacion", {}).get("nifEmpresa"),
    "provincia":     t.get("estacion", {}).get("provincia"),
    "municipio":     t.get("estacion", {}).get("municipio"),
    "direccion":     t.get("estacion", {}).get("direccion"),
    "lat":           t.get("estacion", {}).get("lat"),
    "lon":           t.get("estacion", {}).get("lon"),
} for t in tickets])

for c in ["precio","litros","total","lat","lon"]:
    df_chk[c] = pd.to_numeric(df_chk[c], errors="coerce")

print("Tickets totales:", len(df_chk))
print(df_chk.head(5)) 

print("\nPor empresa:")
print(df_chk.groupby("empresa").size())

print("\nPor usuario:")
print(df_chk.groupby("usuario").size().head(10))

print("\nPor producto:")
print(df_chk.groupby("producto").agg(n=("producto","size"), p_med=("precio","mean")).reset_index())

print("\nPor grupo (marca):")
print(
    df_chk.groupby("est_grupo")
          .agg(tickets=("empresa","size"),
               p_med=("precio","mean"),
               litros=("litros","sum"),
               gasto=("total","sum"))
          .sort_values("gasto", ascending=False)
          .head(10)
)

print("\nTop 10 estaciones por gasto total:")
print(
    df_chk.groupby(["est_id","est_nombre","est_grupo"])
          .agg(tickets=("empresa","size"), gasto=("total","sum"))
          .sort_values("gasto", ascending=False)
          .head(10)
)

Tickets totales: 1350
  empresa    usuario       fecha      hora           metodo   producto  \
0  EMP001  EMP001-U1  2025-04-13  04:23:15  Tarjeta crédito  Gasóleo A   
1  EMP001  EMP001-U1  2025-06-14  00:57:19  Tarjeta crédito  Gasóleo A   
2  EMP001  EMP001-U1  2025-01-03  13:38:09         Efectivo  Gasóleo A   
3  EMP001  EMP001-U1  2025-06-04  00:23:23  Tarjeta crédito  Gasóleo A   
4  EMP001  EMP001-U1  2025-07-26  17:43:18  Tarjeta crédito  Gasóleo A   

   precio  litros  total est_id                  est_nombre  \
0   1.460   17.44  25.46   1492  COOP. LA SIBERIA EXTREMEÑA   
1   1.401   25.64  35.92   6170             E.S SANT ISIDRE   
2   1.446   50.44  72.94   3956                      REPSOL   
3   1.422   23.02  32.73    550                  GLOBAL OIL   
4   1.437   52.21  75.03    451                          BP   

                    est_grupo    est_nif provincia              municipio  \
0  COOP. LA SIBERIA EXTREMEÑA  U40781619   BADAJOZ            TALARRUBIAS   
